In [ ]:
# #audio data exploration
# from scipy.io import wavfile

# sample_rate, samples = wavfile.read("../data/test/audio/clip_ff72c4530.wav")

# from scipy import signal
# import numpy as np
# def log_specgram(audio, sample_rate, window_size=20,
#                  step_size=10, eps=1e-10):
#     nperseg = int(round(window_size * sample_rate / 1e3))
#     noverlap = int(round(step_size * sample_rate / 1e3))
#     freqs, times, spec = signal.spectrogram(audio,
#                                     fs=sample_rate,
#                                     window='hann',
#                                     nperseg=nperseg,
#                                     noverlap=noverlap,
#                                     detrend=False)
#     return freqs, times, np.log(spec.T.astype(np.float32) + eps)

# freqs, times, spectrogram = log_specgram(samples, sample_rate)

# spectrogram.shape

# import librosa

# S = librosa.feature.melspectrogram(samples, sr=sample_rate, n_mels=128)

# S.shape

# log_S = librosa.power_to_db(S, ref=np.max)

# log_S.shape

# mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)

# mfcc.shape

# delta2_mfcc = librosa.feature.delta(mfcc, order=2)

# delta2_mfcc.shape